### Importing the Libraries


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importing the dataset


In [202]:
dataset = pd.read_csv('dataset/dataset.csv', header = None)
dataset.columns = dataset.iloc[0]
dataset = dataset[1:]
dataset = dataset.drop(columns=['Year'])

### Encoding Categorical Data


In [203]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
dataset['District'] = label_encoder.fit_transform(dataset['District'])

### Handling Missing Values


In [204]:
dataset = dataset.apply(pd.to_numeric, errors='coerce')
# dataset.interpolate(method='linear', inplace=True)
# dataset.fillna(method='ffill', inplace=True)
# dataset.fillna(dataset.mode(), inplace=True)
dataset.ffill(inplace=True)
# dataset.fillna(dataset.median(), inplace=True)
# dataset.fillna(dataset.mode(), inplace=True)
# dataset.fillna(dataset.mean(), inplace=True)

dataset['Total Cases'] = dataset.iloc[:, 37:49].sum(axis=1)
dataset = dataset.drop(dataset.columns[37:49], axis=1)

In [205]:
print(dataset)

0    District  Temperature_1  Temperature_2  Temperature_3  Temperature_4  \
1           0      20.000000      22.680000      26.655000      29.470000   
2           0      19.385000      21.440000      26.865000      29.515000   
3           0      20.640000      23.015000      28.825000      30.685000   
4           0      18.301344      20.119196      27.013710      29.800000   
5           0      18.979301      22.850893      25.262500      28.902778   
..        ...            ...            ...            ...            ...   
380        63      17.180000      19.535000      23.945000      25.630000   
381        63      17.235000      20.965000      24.980000      27.810000   
382        63      15.633737      16.521280      24.450269      25.818472   
383        63      15.571102      20.074256      23.375269      26.259583   
384        63      20.700000      19.900000      23.800000      31.900000   

0    Temperature_5  Temperature_6  Temperature_7  Temperature_8  \
1       

### Handling Missing Values : Interpolation

In [206]:
# dataset = dataset.apply(pd.to_numeric, errors='coerce')
# dataset.interpolate(method='linear', inplace=True)
# dataset['Total Cases'] = dataset.iloc[:, 37:50].sum(axis=1)
# dataset = dataset.drop(dataset.columns[37:49], axis=1)

### Test/Train Split


In [207]:
from sklearn.model_selection import train_test_split

X = dataset.iloc[:, 0:-1].values
Y = dataset.iloc[:, -1].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

### Hyperparameter Tuning

#### Bayesian Optimization


In [208]:
# from skopt import BayesSearchCV
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import mean_squared_error

# # Define the estimator
# estimator = AdaBoostRegressor(
#     estimator=DecisionTreeRegressor(random_state=42),
#     random_state=42
# )

# # Set the parameter bounds
# param_bounds = {
#     'n_estimators': (50, 500),
#     'learning_rate': (0.01, 1.0),
#     'estimator__max_depth': (1, 15),
#     'estimator__min_samples_split': (2, 10),
#     'estimator__min_samples_leaf': (1, 5)
# }

# # Initialize Bayesian Optimization
# optimizer = BayesSearchCV(
#     estimator=estimator,
#     search_spaces=param_bounds,
#     n_iter=50,
#     scoring='neg_mean_squared_error',
#     cv=5,
#     random_state=42,
#     verbose=2
# )

# # Perform optimization
# optimizer.fit(X_train, Y_train)

# # Print the best parameters
# print(f"Best parameters: {optimizer.best_params_}")

In [209]:
# # Import necessary libraries
# import warnings
# import numpy as np
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
# from sklearn.metrics import accuracy_score
# from imblearn.over_sampling import SMOTE  # For oversampling

# # Suppress warnings
# warnings.filterwarnings("ignore", category=UserWarning)
# warnings.filterwarnings("ignore", category=FutureWarning)

# # Example dataset (replace this with your dataset)
# from sklearn.datasets import make_classification
# X, Y = make_classification(n_samples=500, n_features=20, n_classes=3,
#                             n_informative=15, n_redundant=5,
#                             weights=[0.05, 0.15, 0.8], random_state=42)

# # Split dataset into training and test sets
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

# # Balance the dataset using SMOTE
# smote = SMOTE(random_state=42)
# X_resampled, Y_resampled = smote.fit_resample(X_train, Y_train)

# # Define the AdaBoost classifier
# adaboost = AdaBoostClassifier(random_state=42)

# # Define the parameter grid for hyperparameter tuning
# param_grid = {
#     'n_estimators': [50, 100, 200],       # Number of estimators
#     'learning_rate': [0.01, 0.1, 0.5, 1], # Learning rate
#     'algorithm': ['SAMME']                # Use SAMME only to avoid warnings
# }

# # Use Stratified K-Fold for cross-validation
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# # Perform GridSearchCV
# grid_search = GridSearchCV(
#     estimator=adaboost,
#     param_grid=param_grid,
#     scoring='accuracy',
#     cv=cv,
#     verbose=2,
#     n_jobs=-1
# )

# # Fit the model with the balanced training data
# grid_search.fit(X_resampled, Y_resampled)

# # Get the best parameters and score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# print("Best Parameters:", best_params)
# print("Best Cross-Validation Accuracy:", best_score)

# # Evaluate the best model on the test data
# best_model = grid_search.best_estimator_
# Y_pred = best_model.predict(X_test)
# test_accuracy = accuracy_score(Y_test, Y_pred)
# print("Test Accuracy with Best Parameters:", test_accuracy)


#### Random Search

In [210]:
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.metrics import mean_squared_error


# # Define the parameter grid for Random Search
# param_grid = {
#     'n_estimators': [50, 100, 200, 300, 500],
#     'learning_rate': np.linspace(0.01, 1.0, 10),
#     'estimator__max_depth': [1, 3, 5, 10, 15],  # Tuning the weak learner
#     'estimator__min_samples_split': [2, 5, 10],  # Weak learner split
#     'estimator__min_samples_leaf': [1, 2, 5]
# }

# # Initialize the weak learner (Decision Tree)
# base_estimator = DecisionTreeRegressor(random_state=42)

# # Initialize the AdaBoost Regressor with the weak learner
# ada_model = AdaBoostRegressor(estimator=base_estimator, random_state=42)

# # Initialize RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     estimator=ada_model,
#     param_distributions=param_grid,
#     n_iter=50,  # Number of random combinations to try
#     scoring='neg_mean_squared_error',  # Evaluation metric
#     cv=5,  # 5-fold cross-validation
#     verbose=1,
#     random_state=42,
#     n_jobs=-1  # Utilize all CPU cores
# )

# # Fit RandomizedSearchCV
# random_search.fit(X_train, Y_train)

# # Get the best parameters and model performance
# print(f"Best parameters: {random_search.best_params_}")
# print(f"Best RMSE (CV): {np.sqrt(-random_search.best_score_)}")

# # Evaluate the best model on the test set
# best_model = random_search.best_estimator_
# y_pred = best_model.predict(X_test)
# test_rmse = mean_squared_error(Y_test, y_pred, squared=False)
# print(f"Test RMSE: {test_rmse}")


### Training Model


In [211]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
#base_estimator = DecisionTreeRegressor(max_depth=15, min_samples_split=2, min_samples_leaf=1)
base_estimator = DecisionTreeRegressor(max_depth=10, min_samples_split=5, min_samples_leaf=5)
# regressor = AdaBoostRegressor(estimator=base_estimator, n_estimators=200, learning_rate=0.01)
# regressor = AdaBoostRegressor()

#Best Parameters: {'algorithm': 'SAMME', 'learning_rate': 1, 'n_estimators': 200}
regressor = AdaBoostRegressor(estimator=base_estimator, n_estimators=200, learning_rate=0.01)

regressor.fit(X_train, Y_train)

# Best parameters: {'n_estimators': 200, 'learning_rate': 0.01, 'estimator__min_samples_split': 10, 'estimator__min_samples_leaf': 5, 'estimator__max_depth': 10}



AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=10,
                                                  min_samples_leaf=5,
                                                  min_samples_split=5),
                  learning_rate=0.01, n_estimators=200)

#### Comparing Values


In [212]:
Y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2, suppress=True)
print(np.concatenate((Y_pred.reshape(len(Y_pred),1), Y_test.reshape(len(Y_test),1)),1))


[[  291.96   336.  ]
 [ 1261.57   493.  ]
 [ 1803.88  6950.  ]
 [ 4581.4   1448.  ]
 [ 4624.    2841.  ]
 [ 1561.     491.  ]
 [  132.19    27.  ]
 [   77.37     1.  ]
 [   51.41    10.  ]
 [ 1654.67   290.  ]
 [ 3620.    3407.  ]
 [  275.5    144.  ]
 [  376.85   109.  ]
 [  376.67    96.  ]
 [ 1135.2    140.  ]
 [  124.9     10.  ]
 [  118.31  5628.  ]
 [  792.33   762.  ]
 [ 1017.11  2224.  ]
 [  142.12    67.  ]
 [  555.4      0.  ]
 [  124.33   318.  ]
 [  382.22     0.  ]
 [  413.6    152.  ]
 [ 1915.     382.  ]
 [ 1017.    4533.  ]
 [  156.11   361.  ]
 [ 2322.6    924.  ]
 [ 1883.6      0.  ]
 [  174.68     0.  ]
 [ 2590.4   6354.  ]
 [ 8564.83  1010.  ]
 [ 1118.14     0.  ]
 [ 2092.2    476.  ]
 [  627.57     0.  ]
 [ 1265.75   529.  ]
 [ 6106.2    335.  ]
 [  140.75     0.  ]
 [ 5085.11  1053.  ]
 [ 3084.    1551.  ]
 [  113.      84.  ]
 [  177.12    17.  ]
 [ 2958.33   739.  ]
 [  246.35    62.  ]
 [  343.33   281.  ]
 [   82.16     4.  ]
 [  959.85  4541.  ]
 [  189.12   

In [213]:
from sklearn.metrics import r2_score
r2_score(Y_test, Y_pred)

-0.1811242076185633

In [214]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true, dtype=float), np.array(y_pred, dtype=float)
    non_zero_indices = y_true != 0
    return np.mean(np.abs((y_true[non_zero_indices] - y_pred[non_zero_indices]) / y_true[non_zero_indices])) * 100


mean_absolute_percentage_error(Y_test, Y_pred)

417.2485288322064